In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.colors as colors
import ipdb
import platform
opj = os.path.join
deb = ipdb.set_trace
%matplotlib inline
import time

# MRI imports
# -----------
import nibabel as nb
import cortex

subject = '999999'

# Define analysis parameters
# --------------------------
with open('../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

# Define cluster/server specific parameters
# -----------------------------------------
if 'aeneas' in platform.uname()[1]:
    base_dir = analysis_info['aeneas_base_folder']
elif 'local' in platform.uname()[1]:
    base_dir = analysis_info['local_base_folder']

fit_model = 'gauss'
deriv_dir = opj(base_dir,'pp_data',subject,fit_model,'deriv')

# Create derivatives flatmaps
# ---------------------------
sign_idx, rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
            non_lin_idx, amp_idx, baseline_idx, cov_idx, x_idx, y_idx = 0,1,2,3,4,5,6,7,8,9,10,11
mask_dir  = 'all'

# Get data and combine hemispheres
deriv_mat=[]
for hemi in ['L','R']:
    deriv_file = nb.load(opj(deriv_dir,mask_dir,"prf_deriv_{hemi}_{mask_dir}_fsaverage.func.gii".format(hemi = hemi, mask_dir = mask_dir)))
    deriv_mat.append(np.array([deriv_file.darrays[i].data for i in range(len(deriv_file.darrays))]))
deriv_mat = np.hstack(deriv_mat)
deriv_mat = deriv_mat.T

ts_mat=[]
ts_file = nb.load(opj(base_dir,'raw_data',subject,'tfMRI_RETBAR1_7T_AP_Atlas_MSMAll_hp2000_clean.dtseries_LR.func_bla_psc_av_fsaverage.gii'))
ts_mat.append(np.array([ts_file.darrays[i].data for i in range(len(ts_file.darrays))]))
ts_mat = np.hstack(ts_mat)

In [2]:
flat = {'camera.altitude':0,
        'camera.azimuth':180,
        'camera.radius':300,
        'surface.{subject}.depth':1,
        'surface.{subject}.unfold':1, # 0 for reset 1 for flat
        'surface.{subject}.specularity':0,
                            }

left_hemi_lateral_inflated = {'camera.altitude':90,
                             'camera.azimuth':90,
                             'camera.radius':300,
                             'surface.{subject}.unfold':0.5, # 0 for reset 1 for flat
                             'surface.{subject}.depth':1,
                              'surface.{subject}.left':True,
                             'surface.{subject}.right':False,
                             'surface.{subject}.specularity':0,
                            }

left_hemi_medial_inflated = {'camera.altitude':90,
                             'camera.azimuth':270,
                             'camera.radius':300,
                             'surface.{subject}.unfold':0.5, # 0 for reset 1 for flat
                             'surface.{subject}.depth':1,
                             'surface.{subject}.left':True,
                             'surface.{subject}.right':False,
                             'surface.{subject}.specularity':0,
                            }
right_hemi_lateral_inflated = {  'camera.altitude':90,
                                 'camera.azimuth':270,
                                 'camera.radius':300,
                                 'surface.{subject}.unfold':0.5, # 0 for reset 1 for flat
                                 'surface.{subject}.depth':1,
                                 'surface.{subject}.left':False,
                                 'surface.{subject}.right':True,
                                 'surface.{subject}.specularity':0,
                                }
right_hemi_medial_inflated = {   'camera.altitude':90,
                                 'camera.azimuth':90,
                                 'camera.radius':300,
                                 'surface.{subject}.unfold':0.5, # 0 for reset 1 for flat
                                 'surface.{subject}.depth':1,
                                 'surface.{subject}.left':False,
                                 'surface.{subject}.right':True,
                                 'surface.{subject}.specularity':0
                             }

dorsal_inflated = {   'camera.altitude':90,
                      'camera.azimuth':180,
                      'camera.radius':300,
                      'surface.{subject}.unfold':0.5, # 0 for reset 1 for flat
                      'surface.{subject}.left':True,
                      'surface.{subject}.depth':1,
                      'surface.{subject}.right':True,
                      'surface.{subject}.specularity':0.2,
                                }

views = ['flat','left_hemi_lateral_inflated','left_hemi_medial_inflated','right_hemi_lateral_inflated','right_hemi_medial_inflated','dorsal_inflated']



In [3]:
cmap = 'BuBkRd_alpha_2D'
# R-square
rsq_data = deriv_mat[:,rsq_idx]
rsq_data = np.nan_to_num(rsq_data)
rsq_data = rsq_data*255.0
rsq_data = rsq_data.astype(np.uint8)

vertex_ts = cortex.Vertex2D(   dim1 = ts_mat, 
                                  dim2 = rsq_data,
                                  description = 'time_course',
                                  subject = 'fsaverage', 
                                  cmap = cmap,
                                  vmin = -2,
                                  vmax = 2,
                                  vmin2 = 0,
                                  vmax2 = 255
                         )

In [4]:
handle = cortex.webgl.show(data = vertex_ts, recache = True)

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 54416


In [36]:
t2 = {'frame':2}

exec("handle._set_view(**{view})".format(view = t2))


Unknown parameter frame!


In [12]:
handle.setFrame.

Exception: undefined is not an object (evaluating 'event.value[0]')

In [10]:
handle.frame

0

In [53]:
# to set manually
# put roi visible not label
# put scale to 0 255

img_dir = '/home/szinte/projects/retino_HCP/post_fit/notebooks/figs/FigureVSS'
type_plot = 'polarity_roi'

views = ['flat','left_hemi_lateral_inflated','left_hemi_medial_inflated','right_hemi_lateral_inflated','right_hemi_medial_inflated','dorsal_inflated']
                              
for view in views:
    exec("handle._set_view(**{view})".format(view = view))
    handle.getImage("{imdir}/{view}_{type_plot}.png".format(imdir=img_dir, view = view, type_plot = type_plot),size = (3840, 2160))